In [ ]:
%%capture
!pip install openai
!pip install evaluate
!pip install rouge_score
!pip install replicate

In [ ]:
import pandas as pd
import openai
import evaluate
import numpy as np
import replicate
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.environ["REPLICATE_API_TOKEN"] = #insert API token here

In [ ]:
valA = pd.read_csv('TaskA-ValidationSet.csv')

In [ ]:
sections = valA['section_header'].unique().tolist()

In [ ]:
def get_al(model, prompt, max_tokens):
  if model == "replicate/llama70b-v2:14ce4448d5e7e9ed0c37745ac46eca157aab09061f0c179ac2b323b5de56552b":
    inp = {"prompt": prompt, "max_new_tokens": max_tokens, "temperature": 0.2}
  else:
    inp = {"prompt": prompt, "max_new_tokens": max_tokens, "temperature": 0.2, "system_prompt": "You are a smart doctor as well as a professional medical scribe with a lot of experience."}

  output = replicate.run(
    model,
    input=inp,
  )
  result = ''.join(output)
  return result

In [ ]:
#first is the base version, second is the chat version
model_lst = ["replicate/llama70b-v2:14ce4448d5e7e9ed0c37745ac46eca157aab09061f0c179ac2b323b5de56552b", "replicate/llama70b-v2-chat:2d19859030ff705a87c746f7e96eea03aefb71f166725aee39692f1476566d48"]
max_tokens = [40]
model_names = {
    "replicate/llama70b-v2:14ce4448d5e7e9ed0c37745ac46eca157aab09061f0c179ac2b323b5de56552b": "llama2_base",
    "replicate/llama70b-v2-chat:2d19859030ff705a87c746f7e96eea03aefb71f166725aee39692f1476566d48": "llama2_chat"
}

In [ ]:
def get_prompt(dia, section):
  prompts = {
    "EXAM": f"Task: I have a doctor-patient dialogue. Please list each body part examined by the doctor along with what was observed as a result for each body part. Please be concise. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\nDialogue: \n{dia} \n\nExaminations:",
    "OTHER_HISTORY": f"Task: I have a doctor-patient dialogue. Please output the patient's medical history and other history. If nothing has changed in the patient's history, say so. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\nDialogue: \n{dia} \n\nPatient's history:",
    "CC": f"Task: I have a doctor-patient dialogue. Please output the patient's chief complaint, which are the reasons/symptoms of why the patient is visiting the doctor. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Chief complaint:",
    "DIAGNOSIS": f"Task: I have a doctor-patient dialogue. Please list the patient's diagnosis. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Diagnosis:",
    "FAM/SOCHX": f"Task: I have a doctor-patient dialogue. Please output the patient's family medical history and the patient's social history. If the history is unknown, say so. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Patient's family history and social history:",
    "MEDICATIONS": f"Task: I have a doctor-patient dialogue. Please list all the medications taken by the patient as part of a doctor's note. If no medications are mentioned, please output None. You are a smart doctor that knows this. \n\n Dialogue: \n{dia} \n\n Medications:",
    "PROCEDURES": f"Task: I have a doctor-patient dialogue. Please output the patient's surgical history, past procedures, current procedures, as well as the outcomes of the procedures/surgeries. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo for naming the procedures/surgeries as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Procedures/surgeries:",
    "ALLERGY": f"Task: I have a doctor-patient dialogue. Please output the patient's allergies. Please use medical terminology and lingo as much as possible. Please output None if there are no allergies. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Allergies:",
    "GENHX": f"Task: I have a doctor-patient dialogue. Please output the patient's history regarding the present illness. That is, output the history of the issue the patient is currently suffering from. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Present illness history:",
    'ROS': f"Task: I have a doctor-patient dialogue. Please output the review of systems (ROS) regarding the patient, that is the signs and/or symptoms which the patient may be experiencing or has experienced and on which part of the body. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo for naming the symptoms and issues as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Review of systems:",
    'PASTMEDICALHX':f"Task: I have a doctor-patient dialogue. Please output the patient's past medical history. If nothing has changed in the patient's history, say so. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Patient's medical history:",
    'PASTSURGICAL':f"Task: I have a doctor-patient dialogue. Please list all the of patient's past surgeries as part of a doctor's note. If no surgeries, please output None. If surgeries are unknown, please output Unknown. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Past surgeries:",
    'DISPOSITION': f"Task: I have a doctor-patient dialogue. Please output the patient's disposition. Please be concise. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Disposition:",
    'EDCOURSE': f"Task: I have a doctor-patient dialogue. The dialogue either mentions what happened in the emergency room (ER) or is currently taking place at the emergency room. Please summarize what happened or is happening at the emergency room. Output what the patient suffered/is suffering from and what the treatments were/are and the dosages if mentioned. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Emergency room summary:",
    'PLAN': f"Task: I have a doctor-patient dialogue. Please list the plan the patient has been following and/or the plan the patient and the doctor agreed on to follow from now on. Please list only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Plan:",
    'LABS': f"Task: I have a doctor-patient dialogue. Please output the results of the lab tests. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Lab results:",
    'ASSESSMENT': f"Task: I have a doctor-patient dialogue. Please list the doctor's medical observations about the patient as well as what the patient mentions about themselves medically. Basically, list everything medical from the assessment. Please only list what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Assessment:",
    'GYNHX': f"Task: I have a doctor-patient dialogue. Please output anything that was mentioned related to gynecology, obstetrics, OB/GYN, pap smears, pregnancy, menstrual cycles (periods), organs related to OB/GYN, results related to OB/GYN, issues related to OB/GYN, history related to OB/GYN, etc. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n OB/GYN:",
    'IMAGING': f"Task: I have a doctor-patient dialogue. Please output any imaging that was done and the results of the imaging. Imaging such as x-ray, EKG, CBC, radiology, ultrasound, etc. Please output only what was mentioned in the conversation and don't add any other new information. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Imaging:",
    'IMMUNIZATIONS': f"Task: I have a doctor-patient dialogue. Please output the patient's immunizations, vaccines, how up-to-date the immunizations are, and the date of when the immunizations were given. Please use medical terminology and lingo as much as possible. You are a smart doctor and a professional medical scribe. \n\n Dialogue: \n{dia} \n\n Immunizations and their dates:",
  }
  return prompts[section]

In [ ]:
section_pred_lst = []
gold_lst = []
for section, max_token in zip(sections, max_tokens):
  valA_al = valA[valA['section_header'] == section]
  dialogue_al = valA_al['dialogue'].tolist()
  gold_lst.append(valA_al['section_text'].tolist())
  model_pred_lst = []
  for model in model_lst:
    fun = get_al
    al_pred_lst = []
    for dia in dialogue_al:
      prompt = get_prompt(dia, section)
      res = fun(model, prompt, max_token)
      al_pred_lst.append(res)
    np.save(f'{model_names[model]}_{section}_preds.npy', al_pred_lst)
    model_pred_lst.append(al_pred_lst)
  section_pred_lst.append(model_pred_lst)

In [ ]:
rouge = evaluate.load('rouge')
section_rouge_scores = []
for section, gold_al in zip(section_pred_lst, gold_lst):
  model_rouge_scores = []
  for preds in section:
    rouge_res = rouge.compute(predictions=preds, references=gold_al)
    model_rouge_scores.append(rouge_res)
  section_rouge_scores.append(model_rouge_scores)

In [ ]:
for section, section_rouge in zip(sections, section_rouge_scores):
  print(f'{section}:\n')
  for i, rouge in enumerate(section_rouge):
    print(f"{model_names[model_lst[i]]}: ", rouge, "\n")